# mVEP ESI N200 Analysis

EEG Source Imaging, or ESI, is a technique to project EEG sensor data onto its corresponding cortical area. As a result, it allows us to get a better sense of the underlying brain activity than just performing sensor-level analysis (Especially due to the fact we had to rotate EEG caps with non-standard techniques for subjects in this study to expose their V5 between electrodes). 

To do this, we can make use of MNE Python's automated co-registration (a way to match the EEG landmarks to the subject's MRI file). Once we have the brain voxel ESI, we can extract and compare specific cortical areas based on FreeSurfer segmented regions. 

Scripts are primarily written by Joshua Kosnoff. I've tried to give proper credit to 3rd party sites where due. In many cases, MNE-based codes were liberally copied from MNE Python tutorials 

## Functions

Lots of functions for automating the process. In most cases of MNE-based processes, the functions were put together from tutorials on their site.

In [1]:
# Native Python packages
import os.path 
import json
import copy

# Third Party Packages
import numpy as np
import pandas as pd
import scipy
from scipy import stats
import matplotlib.pyplot as plt
import mne

# Scripts within repository
from Outlier_Tests import IQR_Outlier_test
from Raincloud_Plot_Maker import statistical_box_and_whisker

# Silence mne logs
mne.set_log_level('ERROR')

# MNE Toolbox functions for mVEP BCI analysis

def auto_coregister(subject, subject_dir, subject_raw, plot_coreg = True, save_file = True, session_num=""):
    
    # Note: Before running this, you need to make the BEM model for the subject. See the my FreeSurfer Protocol 
    # for the terminal commands on how to do this
     
    # Cribbed from https://mne.tools/dev/auto_tutorials/forward/25_automated_coreg.html.
    # Reformatted into a function by Joshua Kosnoff.

    info = mne.io.read_info(fr"{subject_dir}/{subject_raw}")
    plot_kwargs = dict(
        subject=subject,
        subjects_dir=subject_dir,
        surfaces="head",
        dig=True,
        eeg="projected",
        meg=[],
        show_axes=True,
        coord_frame="auto",
    )
    view_kwargs = dict(azimuth=30, elevation=90, distance=0.6, focalpoint=(0.0, 0.0, 0.0))

    fiducials = "auto"  # get fiducials from fsaverage
    
    coreg = mne.coreg.Coregistration(info, subject, subject_dir, fiducials=fiducials)
    coreg.omit_head_shape_points(distance=5.0 / 1000)  # distance is in meters
    coreg.fit_icp(n_iterations=30, nasion_weight=10.0)
    
    # Visualize the coregistration
    if plot_coreg:
        fig = mne.viz.plot_alignment(info, trans=coreg.trans, **plot_kwargs)
        mne.viz.set_3d_view(fig, **view_kwargs)

    # Save the file to the subject directory
    if save_file:
        mne.write_trans(fr'{subject_dir}/{subject}_{session_num}-trans.fif', coreg.trans, overwrite=True)
    
    return

def make_digitization_fif(subject, subject_dir, path_to_vdhr, path_to_RAS, session_num = ""):
    raw = mne.io.read_raw_brainvision(path_to_vdhr,misc=['ECG', 'EMG', 'FootPad'],preload=False)
    mont = mne.channels.read_dig_localite(path_to_RAS)
    raw.set_montage(mont, match_case=False, on_missing='warn')
    raw.save(fr"{subject_dir}/{subject}_{session_num}_eeg.fif", overwrite=True)
    return

def make_inv_operator(epoched_data, subj='default', 
                      subject_dir='default', 
                      coregistration='default', 
                      conductivity = (0.3, 0.006, 0.3), 
                      verbose = 'ERROR',
                      SNR = 3, plot = False):
    
    # Heavily copied from MNE's source imaging tutorial
    
    # Silence all logs (semi-colons added on to functions for good measure)
    mne.utils.use_log_level(verbose)
    mne.set_log_file(verbose)
    
    # If using default head models:
    if subj == 'default':
        path = str(mne.datasets.sample.data_path())
        subj = 'sample'
        subject_dir = path + r'/subjects'
        coregistration = path + r'/MEG/sample/sample_audvis_raw-trans.fif'
        
    elif coregistration == 'default':
        # Open GUI and get it?
        # write it to and load it from the subject_dir
        pass
    
    # Compute Noise Covariance Matrix
    noise_cov = mne.compute_covariance(
        epoched_data, tmax=0., method=['shrunk', 'empirical'], rank=None);
    
    src = mne.setup_source_space(subject=subj, spacing='oct6', add_dist='patch',
                             subjects_dir=subject_dir);
    
    # Make the BEM model    
    bem_fif = coregistration.replace("-trans", "-bem")
    if os.path.isfile(bem_fif): # check to see if the file already exists  
        bem = mne.read_bem_solution(bem_fif)
    else:
        print(f"Writing BEM file for {bem_fif.split(r'/')[-1].split('-')[0]}")
        model = mne.make_bem_model(subject=subj, ico=4,
                                   conductivity=conductivity,
                                   subjects_dir=subject_dir);
        bem = mne.make_bem_solution(model);
        # save the bem model
        mne.write_bem_solution(bem_fif, bem)
    
    # Compute the Forward Solution
    fwd = mne.make_forward_solution(epoched_data.info, trans=coregistration, src=src, bem=bem,
                                    meg=False, eeg=True, mindist=5.0, n_jobs=None,);

    # Get Inverse Opterator
    evoked = epoched_data.average();
    
    inv_operator = mne.minimum_norm.make_inverse_operator(
        info=evoked.info, 
        forward = fwd, 
        noise_cov = noise_cov);

    # set signal-to-noise ratio (SNR) to compute regularization parameter (λ²)
    lambda2 = 1. / SNR ** 2
    
    return src, inv_operator, lambda2


## Load in the preprocessed EEG data

In the *mVEP EEG Sensor Domain Analysis_Final* notebook, we preprocessed the raw EEG data. Let's re-use the preprocessed data by loading in the pickle file

In [2]:
# Load in the cleaned dataframe (see:  notebook)
preprocessed_df = pd.read_pickle(f"Analysis_Rerun.pkl")
preprocessed_df

,subject,condition,data_epoch,hit
0,Subj01,Non-Modulated,"<EpochsArray | 1 events (all good), -0.2 – 0....",True
1,Subj01,Non-Modulated,"<EpochsArray | 1 events (all good), -0.2 – 0....",True
2,Subj01,Non-Modulated,"<EpochsArray | 1 events (all good), -0.2 – 0....",True
3,Subj01,Non-Modulated,"<EpochsArray | 1 events (all good), -0.2 – 0....",True
4,Subj01,Non-Modulated,"<EpochsArray | 1 events (all good), -0.2 – 0....",True
...,...,...,...,...
5157,Subj25,tFUS-GP,"<EpochsArray | 1 events (all good), -0.2 – 0....",True
5158,Subj25,tFUS-GP,"<EpochsArray | 1 events (all good), -0.2 – 0....",True
5159,Subj25,tFUS-GP,"<EpochsArray | 1 events (all good), -0.2 – 0....",True
5160,Subj25,tFUS-GP,"<EpochsArray | 1 events (all good), -0.2 – 0....",True


In [3]:
print(len(np.unique(preprocessed_df.subject)))

25


## DATA PATHS

NOTE: Some subjects had errors with the segmented MRI files. 
There does not seem to an easy fix for this (though it looks possible with manual editing of their 3D render) --> 
For now, using the default MRI brain model in their place. 

ALSO NOTE: Since testing data generally yeilded high error, I analyzed the neural data from the **training** sets.
This should (theorectically) increase the likelihood that the subjects were looking at the proper letter for each
trial (especially for the earlier cases, where we did not use an indicator in the testing session)


In [4]:
###################################################
# CHANGE THESE PATHS - the relative paths for box should be the same. You may need to change
# the relative FreeSurfer paths if you have different nomenclature or organization methods

path_to_file_paths = r"file_paths_remapped.json"
path_to_box = r"/Users/jkosnoff/Library/CloudStorage/Box-Box/Joshua-Speller-Paper/Data/EEG_Data/"

path_to_freesurfer = path_to_box + r'/Freesurfer' # r"/Users/jkosnoff/Documents/Freesurfer"
path_to_freesurfer = r"/Users/jkosnoff/Docs/Freesurfer"
###################################################

## EEG Source Imaging

Now we're ready to perform the EEG Source Imaging. As mentioned earlier, we need to align the EEG landmarks (embedded in the *RAS_file*) with the subject's MRI (*subject_dir* / *head_model*). The code below will create the eeg-digization and co-registration files if they do not already exist (but these can take a long time to make, so it will check if it can load pre-made files first). 

For this study, the three primary areas of interest for theta power analysis are the left (*lh*) V5 (*MT_exvivo*; part of Broddman's labeled dictionary *BA_exvivo*), the superior parietal lobe (*superiorparietal*; Desikan-Killiany Atlas *aparc*), and the inferior temporal gyrus (*inferiortemporal*, Desikan-Killiany Atlas *aparc*). We can extract the average activity in each each region and store it to the designated DataFrame.

In [5]:
# The paracellation dictionary that the labels are in
# Use BA_exvivo.thresh for smaller region
aparcs = ['BA_exvivo', 'aparc', 'aparc']


key_mappings = {"Decoupled-Sham": "Decoupled-Sham", "Non-Modulated":"Non-Modulated", 
               "tFUS-GC":"tFUS", "tFUS-GP": "US-Control"}

# The FreeSurfer labels of interest
BA_regions = ["MT_exvivo-lh",
    "superiorparietal-lh",
    "inferiortemporal-lh"]

with open(path_to_file_paths) as f:
    file_paths = json.load(f)

for aparc, FS_region in zip(aparcs, BA_regions):
    
    # Initialize a new data frame
    ESI_df = pd.DataFrame(columns = ["subject", "condition", "data", "scan_num", "order"])

    for subject in np.unique(preprocessed_df.subject):
        
        # Step 0: Get the relevant paths
        head_model = file_paths[subject]["head_model"]
        subject_dir =  path_to_freesurfer + file_paths[subject]["subject_dir"]
        RAS_file = subject_dir + file_paths[subject]["RAS"]
        session_num = RAS_file.split(r"/")[-2]
        
        # which vhdr file doesn't actually matter since we already have the data in the dataframe
        # we just need to grab the electrode information present in the file (which should be the same
        # across conditions)
        vhdr_file = path_to_box + subject + "r/" + list(file_paths[subject]["vhdr_paths_training"].values())[0]

        # Step 1: Load (or make) the eeg-digitization fif
        subject_digi_fif = head_model + "_" + session_num + "_eeg.fif"

        if not os.path.isfile(subject_dir + r"/" + subject_digi_fif): # check to see if the file already exists
            make_digitization_fif(subject=head_model, subject_dir=subject_dir,
                              path_to_vdhr=vhdr_file, path_to_RAS=RAS_file, session_num=session_num)

        # Step 2: Load (or make) the coregistration fif
        subject_tran_fif = head_model  + "_" + session_num + "-trans.fif"
        if not os.path.isfile(subject_dir + r"/" + subject_tran_fif): #check to see if file already exists
            auto_coregister(subject=head_model, subject_dir=subject_dir, 
                            subject_raw=subject_digi_fif, save_file=True, plot_coreg=False, session_num=session_num)
            
        # Step 3: Load the EEG data               
        for condition in list(file_paths[subject]["vhdr_paths_training"].keys()):
            
            # Since the ESI analysis uses each epoch scan as a trial, the sample size is already much higher 
            # than that of the behavioral outcome data. In an attempt to avoid over-biasing towards certain 
            # subjects, if one subject as data sets for a condition, only consider the first one. 
            
            if condition[-1] not in ["2", "3", "4", "5", "6", "7", "8", "9"]:
                epoch = mne.concatenate_epochs(preprocessed_df["data_epoch"].loc[(preprocessed_df.subject == subject) 
                                                                                 & (preprocessed_df.condition == condition)
                                                                                 & (preprocessed_df.hit == True)
                                                                               ].to_list())
                
                # The MNE inverase operator function requires a data reference.
                epoch.set_eeg_reference('average', projection=True)
                
                # Step 4: Make the inverse operator
                src, inv_operator, lambda2 = make_inv_operator(
                        epoched_data= epoch, 
                        subj=head_model, 
                        subject_dir=subject_dir, 
                        coregistration=subject_dir + r"/" + subject_tran_fif)

                # Step 5: Extract source imaging time series
                # Get the FreeSurfer Labels for the specified region and parcellation
                labels = mne.read_labels_from_annot(head_model, parc=aparc, subjects_dir=subject_dir, regexp=FS_region)

                stcs = mne.minimum_norm.apply_inverse_epochs(epoch, 
                                                             inv_operator,
                                                             lambda2=lambda2,
                                                             method='MNE');  # MNE, or dSPM, sLORETA
                for i, stc in enumerate(stcs):
                    ts = mne.extract_label_time_course(stcs = stc, 
                                                        labels = labels,
                                                        src=src)

                    ESI_df.loc[len(ESI_df)] = {"subject": subject, "condition": condition, 
                                               "data": ts.flatten(), "scan_num": i, 
                                               "order": file_paths[subject]['order'][key_mappings[condition]]}
    
    # Save the data to a dataframe / pickle
    ESI_df.to_pickle(f"Analysis_Rerun_{FS_region}.pkl")     

/var/folders/yy/4qhb9qd51zxgxb1p39j8bnnm0000gn/T/ipykernel_93638/1942503868.py:79: RuntimeWarning: Log entries will be appended to the file. Use overwrite=False to avoid this message in the future.
  mne.set_log_file(verbose)
/var/folders/yy/4qhb9qd51zxgxb1p39j8bnnm0000gn/T/ipykernel_93638/1942503868.py:79: RuntimeWarning: Log entries will be appended to the file. Use overwrite=False to avoid this message in the future.
  mne.set_log_file(verbose)
/var/folders/yy/4qhb9qd51zxgxb1p39j8bnnm0000gn/T/ipykernel_93638/1942503868.py:79: RuntimeWarning: Log entries will be appended to the file. Use overwrite=False to avoid this message in the future.
  mne.set_log_file(verbose)
/var/folders/yy/4qhb9qd51zxgxb1p39j8bnnm0000gn/T/ipykernel_93638/1942503868.py:79: RuntimeWarning: Log entries will be appended to the file. Use overwrite=False to avoid this message in the future.
  mne.set_log_file(verbose)
/var/folders/yy/4qhb9qd51zxgxb1p39j8bnnm0000gn/T/ipykernel_93638/1942503868.py:79: RuntimeWarnin

/var/folders/yy/4qhb9qd51zxgxb1p39j8bnnm0000gn/T/ipykernel_93638/1942503868.py:79: RuntimeWarning: Log entries will be appended to the file. Use overwrite=False to avoid this message in the future.
  mne.set_log_file(verbose)
/var/folders/yy/4qhb9qd51zxgxb1p39j8bnnm0000gn/T/ipykernel_93638/1942503868.py:79: RuntimeWarning: Log entries will be appended to the file. Use overwrite=False to avoid this message in the future.
  mne.set_log_file(verbose)
/var/folders/yy/4qhb9qd51zxgxb1p39j8bnnm0000gn/T/ipykernel_93638/1942503868.py:79: RuntimeWarning: Log entries will be appended to the file. Use overwrite=False to avoid this message in the future.
  mne.set_log_file(verbose)
/var/folders/yy/4qhb9qd51zxgxb1p39j8bnnm0000gn/T/ipykernel_93638/1942503868.py:79: RuntimeWarning: Log entries will be appended to the file. Use overwrite=False to avoid this message in the future.
  mne.set_log_file(verbose)
/var/folders/yy/4qhb9qd51zxgxb1p39j8bnnm0000gn/T/ipykernel_93638/1942503868.py:79: RuntimeWarnin

/var/folders/yy/4qhb9qd51zxgxb1p39j8bnnm0000gn/T/ipykernel_93638/1942503868.py:79: RuntimeWarning: Log entries will be appended to the file. Use overwrite=False to avoid this message in the future.
  mne.set_log_file(verbose)
/var/folders/yy/4qhb9qd51zxgxb1p39j8bnnm0000gn/T/ipykernel_93638/1942503868.py:79: RuntimeWarning: Log entries will be appended to the file. Use overwrite=False to avoid this message in the future.
  mne.set_log_file(verbose)
/var/folders/yy/4qhb9qd51zxgxb1p39j8bnnm0000gn/T/ipykernel_93638/1942503868.py:79: RuntimeWarning: Log entries will be appended to the file. Use overwrite=False to avoid this message in the future.
  mne.set_log_file(verbose)
/var/folders/yy/4qhb9qd51zxgxb1p39j8bnnm0000gn/T/ipykernel_93638/1942503868.py:79: RuntimeWarning: Log entries will be appended to the file. Use overwrite=False to avoid this message in the future.
  mne.set_log_file(verbose)
/var/folders/yy/4qhb9qd51zxgxb1p39j8bnnm0000gn/T/ipykernel_93638/1942503868.py:79: RuntimeWarnin

/var/folders/yy/4qhb9qd51zxgxb1p39j8bnnm0000gn/T/ipykernel_93638/1942503868.py:79: RuntimeWarning: Log entries will be appended to the file. Use overwrite=False to avoid this message in the future.
  mne.set_log_file(verbose)
/var/folders/yy/4qhb9qd51zxgxb1p39j8bnnm0000gn/T/ipykernel_93638/1942503868.py:79: RuntimeWarning: Log entries will be appended to the file. Use overwrite=False to avoid this message in the future.
  mne.set_log_file(verbose)
/var/folders/yy/4qhb9qd51zxgxb1p39j8bnnm0000gn/T/ipykernel_93638/1942503868.py:79: RuntimeWarning: Log entries will be appended to the file. Use overwrite=False to avoid this message in the future.
  mne.set_log_file(verbose)
/var/folders/yy/4qhb9qd51zxgxb1p39j8bnnm0000gn/T/ipykernel_93638/1942503868.py:79: RuntimeWarning: Log entries will be appended to the file. Use overwrite=False to avoid this message in the future.
  mne.set_log_file(verbose)
/var/folders/yy/4qhb9qd51zxgxb1p39j8bnnm0000gn/T/ipykernel_93638/1942503868.py:79: RuntimeWarnin

/var/folders/yy/4qhb9qd51zxgxb1p39j8bnnm0000gn/T/ipykernel_93638/1942503868.py:79: RuntimeWarning: Log entries will be appended to the file. Use overwrite=False to avoid this message in the future.
  mne.set_log_file(verbose)
/var/folders/yy/4qhb9qd51zxgxb1p39j8bnnm0000gn/T/ipykernel_93638/1942503868.py:79: RuntimeWarning: Log entries will be appended to the file. Use overwrite=False to avoid this message in the future.
  mne.set_log_file(verbose)
/var/folders/yy/4qhb9qd51zxgxb1p39j8bnnm0000gn/T/ipykernel_93638/1942503868.py:79: RuntimeWarning: Log entries will be appended to the file. Use overwrite=False to avoid this message in the future.
  mne.set_log_file(verbose)
/var/folders/yy/4qhb9qd51zxgxb1p39j8bnnm0000gn/T/ipykernel_93638/1942503868.py:79: RuntimeWarning: Log entries will be appended to the file. Use overwrite=False to avoid this message in the future.
  mne.set_log_file(verbose)
/var/folders/yy/4qhb9qd51zxgxb1p39j8bnnm0000gn/T/ipykernel_93638/1942503868.py:79: RuntimeWarnin

/var/folders/yy/4qhb9qd51zxgxb1p39j8bnnm0000gn/T/ipykernel_93638/1942503868.py:79: RuntimeWarning: Log entries will be appended to the file. Use overwrite=False to avoid this message in the future.
  mne.set_log_file(verbose)
/var/folders/yy/4qhb9qd51zxgxb1p39j8bnnm0000gn/T/ipykernel_93638/1942503868.py:79: RuntimeWarning: Log entries will be appended to the file. Use overwrite=False to avoid this message in the future.
  mne.set_log_file(verbose)
/var/folders/yy/4qhb9qd51zxgxb1p39j8bnnm0000gn/T/ipykernel_93638/1942503868.py:79: RuntimeWarning: Log entries will be appended to the file. Use overwrite=False to avoid this message in the future.
  mne.set_log_file(verbose)
/var/folders/yy/4qhb9qd51zxgxb1p39j8bnnm0000gn/T/ipykernel_93638/1942503868.py:79: RuntimeWarning: Log entries will be appended to the file. Use overwrite=False to avoid this message in the future.
  mne.set_log_file(verbose)
/var/folders/yy/4qhb9qd51zxgxb1p39j8bnnm0000gn/T/ipykernel_93638/1942503868.py:79: RuntimeWarnin

/var/folders/yy/4qhb9qd51zxgxb1p39j8bnnm0000gn/T/ipykernel_93638/1942503868.py:79: RuntimeWarning: Log entries will be appended to the file. Use overwrite=False to avoid this message in the future.
  mne.set_log_file(verbose)
/var/folders/yy/4qhb9qd51zxgxb1p39j8bnnm0000gn/T/ipykernel_93638/1942503868.py:79: RuntimeWarning: Log entries will be appended to the file. Use overwrite=False to avoid this message in the future.
  mne.set_log_file(verbose)
/var/folders/yy/4qhb9qd51zxgxb1p39j8bnnm0000gn/T/ipykernel_93638/1942503868.py:79: RuntimeWarning: Log entries will be appended to the file. Use overwrite=False to avoid this message in the future.
  mne.set_log_file(verbose)
/var/folders/yy/4qhb9qd51zxgxb1p39j8bnnm0000gn/T/ipykernel_93638/1942503868.py:79: RuntimeWarning: Log entries will be appended to the file. Use overwrite=False to avoid this message in the future.
  mne.set_log_file(verbose)
/var/folders/yy/4qhb9qd51zxgxb1p39j8bnnm0000gn/T/ipykernel_93638/1942503868.py:79: RuntimeWarnin

In [6]:
condition

'Decoupled-Sham'

In [7]:
ESI_df.head()

,subject,condition,data,scan_num,order
0,Subj01,tFUS-GC,"[6.25844961958003e-12, 4.4786324621168955e-12,...",0,2
1,Subj01,tFUS-GC,"[4.089245422373442e-12, 1.9210146718524333e-11...",1,2
2,Subj01,tFUS-GC,"[5.6560526907891434e-12, 7.1995791125579404e-1...",2,2
3,Subj01,tFUS-GC,"[3.6170207019923347e-12, 1.7559302076767892e-1...",3,2
4,Subj01,tFUS-GC,"[4.152409647167458e-12, 1.70624367021901e-11, ...",4,2


## Data Analysis

Because we're using repeated samples, we can't use the 'usual suspects' in neural data analysis (i.e., Kruskal Wallis / ANOVA, Wilcoxon / t-test) immediately. Instead, we need to fit the data to linear mixed effect models and treat the subjects as random effect variables. This will be done in R, so let's extract the N200 frequency band power for each band and export it to a csv. Once the data is fit to a linear mixed effect model, we can run z-tests on the model itself.

We will use Morlet Wavelet tranformations to get the associated time-frequency band powers and take the mean over the window (100 - 250 ms). 

We will also use the IQR test to remove outliers for each condition from the power analysis. 

In [3]:
window = range(30, 46) # 100 - 250 ms post simulus onet


# Superior parietal beta, theta, alpha all significantly different
def return_power(array, freqs =  np.arange(12, 31, 1), mark_outliers = True, window = window):
    
    n_cycles = freqs / (2 * freqs[0]) # Morlet wavelet cycles; based on MNE's tutorial

    power = mne.time_frequency.tfr_array_morlet(
                np.expand_dims(array, 1), sfreq = 100.0, 
                freqs = freqs, n_cycles = n_cycles, output='power')[:,:,:,window].mean(axis=-1).mean(axis=-1).mean(axis=-1)
    
    if mark_outliers:
        # Marks outliers as NaNs
        
        power[np.array(IQR_Outlier_test(power))] = np.nan
    
        # power[double_mads_from_median(power) >= 3.5] = np.nan
    
    return power


FS_regions = ['MT_exvivo-lh', 'superiorparietal-lh', 'inferiortemporal-lh']
regions = ['V5', 'SP', 'IT']

for FS_region, region in zip(FS_regions, regions):

    # Load in Data Frame
    df = pd.read_pickle(f"Analysis_Rerun_{FS_region}.pkl")

    # Filter data from 1 - 40 Hz (this should have already been done in the preprocessing, but just in case autoreject / ICA removal messed something up :) ) 
    df["filtered_data"] = list(mne.filter.filter_data(np.vstack(df["data"].values), sfreq=100.0, l_freq=1, h_freq=40))

    # Z-Score based on 200 ms baseline
    df["filtered_data"] =  df["filtered_data"].apply(lambda x: ((x - x[0:20].mean()) / x[0:20].std()) )

    # Initialize Power Column
    df['power'] = np.nan

    freqs = [np.arange(4, 8, 1),  np.arange(8, 12, 1),  np.arange(12, 30, 1), np.arange(30, 40, 1)]
    names = ["theta", "alpha", "beta", "gamma"]

    for freq, name in zip(freqs, names):
        my_df = copy.deepcopy(df)
        for cond in np.unique(df.condition):
#             my_df["power"].loc[my_df.condition == cond] = return_power(np.vstack(my_df['filtered_data'].loc[my_df.condition == cond]), 
#                                                                        freqs = freq)
            condition_mask = (my_df['condition'] == cond)
            my_df.loc[condition_mask, 'power'] = return_power(np.vstack(my_df.loc[condition_mask, 'filtered_data']), freqs=freq)
        
        # Drop the nans (aka drop outliers)
        my_df = my_df.dropna()

        # Print to csv file
        my_df.to_csv(f"/Users/jkosnoff/Downloads/Analysis_Rerun_{region}_{name}_powers-IQR.csv")

In [12]:
import session_info
session_info.show()